This file merges multiple datasets together and processes the data

In [1]:
import pandas as pd
import numpy as np

### Import ACS Data

In [2]:
from ACSDataImport import getS1903, getS1501, getDP02, getB17001, getS0101

In [3]:
s1903 = getS1903()
s1501 = getS1501()
dp02 = getDP02()
b17001 = getB17001()
s0101 = getS0101()

### Clean ACS Data

In [4]:
##### Median Income
s1903_table = s1903.loc[: , ['GEO_ID', 'S1903_C03_001E']]
## Replace '+' and ',' with empty string
s1903_table['S1903_C03_001E'] = (s1903_table['S1903_C03_001E'].str.replace(r'[\+,]', '', regex=True))
## Covert column to numeric
s1903_table['S1903_C03_001E'] = pd.to_numeric(s1903_table['S1903_C03_001E'], errors='coerce')
## Keep rows where income is over 0
s1903_table = s1903_table[s1903_table['S1903_C03_001E'] > 0]
# s1903_table.describe() # used to confirm max house value returns as 250000
# s1903_table.dtypes    # used to confirm column types

In [5]:
##### Educational Attainment
s1501_table = s1501.loc[: , ['GEO_ID', 'S1501_C01_001E', 'S1501_C01_002E', 'S1501_C01_003E', 'S1501_C01_004E', 'S1501_C01_005E', 'S1501_C01_006E', 'S1501_C01_007E', 'S1501_C01_008E', 'S1501_C01_009E', 'S1501_C01_010E', 'S1501_C01_011E', 'S1501_C01_012E', 'S1501_C01_013E', 'S1501_C01_014E', 'S1501_C01_015E']]

# Merge into the following groups:
# 'S1501_C01_001E' - Total (18-24)
# 'S1501_C01_006E' - Total (25+)
s1501_table['001E_006E'] = s1501_table[['S1501_C01_001E', 'S1501_C01_006E']].apply(pd.to_numeric).sum(axis=1)

# 'S1501_C01_002E' - Less than high school (18-24)
# 'S1501_C01_007E' - Less than 9th (25+)
s1501_table['002E_007E'] = s1501_table[['S1501_C01_002E', 'S1501_C01_007E']].apply(pd.to_numeric).sum(axis=1)

# 'S1501_C01_008E' - 9th to 12th, no diploma (25+)
s1501_table['008E'] = s1501_table['S1501_C01_008E'].apply(pd.to_numeric)

# 'S1501_C01_003E' - High school graduate (18-24)
# 'S1501_C01_009E' - High school graduate (25+)
s1501_table['003E_009E'] = s1501_table[['S1501_C01_003E', 'S1501_C01_009E']].apply(pd.to_numeric).sum(axis=1)

# 'S1501_C01_004E' - Some college or associates degree (18-24)
# 'S1501_C01_010E' - Some college, no degree (25+)
# 'S1501_C01_011E' - Associates Degree (25+)
s1501_table['004E_010E_011E'] = s1501_table[['S1501_C01_004E', 'S1501_C01_010E', 'S1501_C01_011E']].apply(pd.to_numeric).sum(axis=1)

# 'S1501_C01_005E' - Bachelor's degree or higher (18-24)
# 'S1501_C01_012E' - Bachelor's degree (25+)
s1501_table['005E_012E'] = s1501_table[['S1501_C01_005E', 'S1501_C01_012E']].apply(pd.to_numeric).sum(axis=1)

# 'S1501_C01_013E' - graduate or professional degree (25+)
s1501_table['013E'] = s1501_table['S1501_C01_013E'].apply(pd.to_numeric)

s1501_table = s1501_table[['GEO_ID', '001E_006E', '002E_007E', '008E', '003E_009E', '004E_010E_011E', '005E_012E', '013E']]
# s1501_table.describe()
# s1501_table.dtypes    # used to confirm column types


In [6]:
##### Household - Total, Married-couple, Co-habitating couple, Male Householder no spouse, Female householder no spouse
dp02_table = dp02.loc[: , ['GEO_ID', 'DP02_0001E', 'DP02_0002E', 'DP02_0004E', 'DP02_0006E', 'DP02_0010E']]
## Covert column to numberic
dp02_table[['DP02_0001E', 'DP02_0002E', 'DP02_0004E', 'DP02_0006E', 'DP02_0010E']] = dp02_table[['DP02_0001E', 'DP02_0002E', 'DP02_0004E', 'DP02_0006E', 'DP02_0010E']].apply(pd.to_numeric, errors='coerce')
# dp02_table.describe()
# dp02_table.dtypes # used to confirm column types
dp02_table

,GEO_ID,DP02_0001E,DP02_0002E,DP02_0004E,DP02_0006E,DP02_0010E
0,1400000US08001007801,1332,276,153,391,512
1,1400000US08001007802,1517,406,211,581,319
2,1400000US08001007900,2264,764,347,595,558
3,1400000US08001008000,2025,685,151,488,701
4,1400000US08001008100,770,114,129,243,284
...,...,...,...,...,...,...
1442,1400000US08123002300,2495,1516,47,482,450
1443,1400000US08123002501,1952,1222,92,201,437
1444,1400000US08123002502,2604,1656,144,379,425
1445,1400000US08125963100,1731,985,72,216,458


In [7]:
##### Household2 - Male Total, Female Total, Never married, now marred except separated, separated, widowed, divorced
dp02_table2 = dp02.loc[: , ['GEO_ID', 'DP02_0025E', 'DP02_0031E', 'DP02_0026E', 'DP02_0027E', 'DP02_0028E', 'DP02_0029E', 'DP02_0030E', 'DP02_0032E', 'DP02_0033E', 'DP02_0034E', 'DP02_0035E', 'DP02_0036E']]
## Covert column to numberic
dp02_table2[['DP02_0026E', 'DP02_0027E', 'DP02_0028E', 'DP02_0029E', 'DP02_0030E', 'DP02_0032E', 'DP02_0033E', 'DP02_0034E', 'DP02_0035E', 'DP02_0036E']] = dp02_table2[['DP02_0026E', 'DP02_0027E', 'DP02_0028E', 'DP02_0029E', 'DP02_0030E', 'DP02_0032E', 'DP02_0033E', 'DP02_0034E', 'DP02_0035E', 'DP02_0036E']].apply(pd.to_numeric, errors='coerce')
# dp02_table2.describe()
# dp02_table2.dtypes # used to confirm column types

In [8]:
##### Household3 - Total population, Native born, foreign born
dp02_table3 = dp02.loc[: , ['GEO_ID', 'DP02_0088E', 'DP02_0089E', 'DP02_0094E']]
## Covert column to numberic
dp02_table3[['DP02_0088E', 'DP02_0089E', 'DP02_0094E']] = dp02_table3[['DP02_0088E', 'DP02_0089E', 'DP02_0094E']].apply(pd.to_numeric, errors='coerce')
# dp02_table3.describe()
# dp02_table3.dtypes # used to confirm column types

In [9]:
##### Household4 - Civilian Pop over 18, Civilian veterans
dp02_table4 = dp02.loc[: , ['GEO_ID', 'DP02_0069E', 'DP02_0070E']]
## Covert column to numberic
dp02_table4[['DP02_0069E', 'DP02_0070E']] = dp02_table4[['DP02_0069E', 'DP02_0070E']].apply(pd.to_numeric, errors='coerce')
# dp02_table4.describe()
# dp02_table4.dtypes # used to confirm column types

In [10]:
##### Poverty - below poverty level, total
b17001_table = b17001.loc[:, ['GEO_ID', 'B17001_001E' , 'B17001_002E']]
b17001_table[['B17001_001E' , 'B17001_002E']] = b17001_table[['B17001_001E' , 'B17001_002E']].apply(pd.to_numeric, errors = 'coerce')
# b17001_table.describe()
# b17001_table.dtypes # used to confirm column types

In [11]:
##### Age - Total population, Under 5 years, 5 to 9 years, 10 to 14 years, 15 to 19 years, 20 to 24 years, 25 to 29 years, 30 to 34 years, 35 to 39 years, 40 to 44 years, 45 to 49 years, 50 to 54 years, 55 to 59 years, 60 to 64 years, 65 to 69 years, 70 to 74 years, 75 to 79 years, 80 to 84 years, 85 years and over
s0101_table = s0101.loc[: , ['GEO_ID', 'S0101_C01_001E', 'S0101_C01_002E', 'S0101_C01_003E', 'S0101_C01_004E', 'S0101_C01_005E', 'S0101_C01_006E', 'S0101_C01_007E', 'S0101_C01_008E', 'S0101_C01_009E', 'S0101_C01_010E', 'S0101_C01_011E', 'S0101_C01_012E', 'S0101_C01_013E', 'S0101_C01_014E', 'S0101_C01_015E', 'S0101_C01_016E', 'S0101_C01_017E', 'S0101_C01_018E', 'S0101_C01_019E']]
s0101_table[['S0101_C01_001E', 'S0101_C01_002E', 'S0101_C01_003E', 'S0101_C01_004E', 'S0101_C01_005E', 'S0101_C01_006E', 'S0101_C01_007E', 'S0101_C01_008E', 'S0101_C01_009E', 'S0101_C01_010E', 'S0101_C01_011E', 'S0101_C01_012E', 'S0101_C01_013E', 'S0101_C01_014E', 'S0101_C01_015E', 'S0101_C01_016E', 'S0101_C01_017E', 'S0101_C01_018E', 'S0101_C01_019E']] = s0101_table[['S0101_C01_001E', 'S0101_C01_002E', 'S0101_C01_003E', 'S0101_C01_004E', 'S0101_C01_005E', 'S0101_C01_006E', 'S0101_C01_007E', 'S0101_C01_008E', 'S0101_C01_009E', 'S0101_C01_010E', 'S0101_C01_011E', 'S0101_C01_012E', 'S0101_C01_013E', 'S0101_C01_014E', 'S0101_C01_015E', 'S0101_C01_016E', 'S0101_C01_017E', 'S0101_C01_018E', 'S0101_C01_019E']].apply(pd.to_numeric, errors = 'coerce')
# s0101_table.describe()
# s0101_table.dtypes # used to confirm column types

### Create Bins for ACS

In [12]:
##### Median Income
## Income (S1903_C03_001E) > median income (s1903_medIncome) - create flag where 1 = True (income >= to than median), 0 = False (income < than median))
s1903_medIncome = np.median(s1903_table['S1903_C03_001E'])

#Binary
s1903_table['householdIncomeAboveMedian'] = (s1903_table['S1903_C03_001E'] >= s1903_medIncome).astype(int)
print(f"Median Household Income ($): {s1903_medIncome}")

#Very Low/Low/Medium/High/Very High bins
s1903_table['householdIncomeBins'] = pd.qcut(s1903_table['S1903_C03_001E'], q =5, labels = ['Very Low','Low', 'Medium', 'High', 'Very High'])
# s1903_table.head()

Median Household Income ($): 90811.0


In [13]:
##### Educational Attainment
# '001E_006E' - Total
# '002E_007E' - Less than high school
# '008E' - High school, no diploma (25+)
# '003E_009E' - High school graduate 
# '004E_010E_011E' - Some college, no degree or Associates Degree
# '005E_012E' - Bachelor's degree or higher
# '013E' - Graduate or professional degree

# Percents
s1501_table['lessThanHS_P'] = round((s1501_table['002E_007E'] / s1501_table['001E_006E'])*100, 2)
s1501_table['HSNoDiploma_P'] = round((s1501_table['008E'] / s1501_table['001E_006E'])*100, 2)
s1501_table['HSGrad_P'] = round((s1501_table['003E_009E'] / s1501_table['001E_006E'])*100, 2)
s1501_table['SomeCollegeNoDeg_P'] = round((s1501_table['004E_010E_011E'] / s1501_table['001E_006E'])*100, 2)
s1501_table['BachOrHigher_P'] = round((s1501_table['005E_012E'] / s1501_table['001E_006E'])*100, 2)
s1501_table['GradOrProf_P'] = round((s1501_table['013E'] / s1501_table['001E_006E'])*100, 2)

# No HS vs HS Grad Percent
s1501_table['noHSDegree_P'] = round(((s1501_table['002E_007E'] + s1501_table['008E']) / s1501_table['001E_006E'])*100, 2)
s1501_table['hasHSDegree_P'] = round(((s1501_table['003E_009E'] + s1501_table['004E_010E_011E'] + s1501_table['005E_012E'] + s1501_table['013E']) / s1501_table['001E_006E'])*100, 2) 
s1501_table.head()

# Very low/Low/Medium/High/Very High on hasHSDegree_P
s1501_table['hsDegreeBins'] = pd.qcut(s1501_table['hasHSDegree_P'], q =5, labels = ['Very Low','Low', 'Medium', 'High', 'Very High'])
# s1501_table.head()


In [14]:
##### Household1
# 'DP02_0001E' - Total
# 'DP02_0002E' - Married-couple households
# 'DP02_0004E' - Cohabiting couple households
# 'DP02_0006E' - Male householder, no spouse
# 'DP02_0010E' - Female householder, no spouse

# Percents 
dp02_table['married_P'] = round((dp02_table['DP02_0002E'] / dp02_table['DP02_0001E'])*100, 2)   #Based on distribution within the geo_id, not within the Married set. (row)
dp02_table['cohabiting_P'] = round((dp02_table['DP02_0004E'] / dp02_table['DP02_0001E'])*100, 2)    #Based on distribution within the geo_id, not within the cohabiting set. 
dp02_table['maleHouseholder_P'] = round((dp02_table['DP02_0006E'] / dp02_table['DP02_0001E'])*100, 2)   #Based on distribution within the geo_id, not within the Male set. 
dp02_table['femaleHouseholder_P'] = round((dp02_table['DP02_0010E'] / dp02_table['DP02_0001E'])*100, 2) #Based on distribution within the geo_id, not within the Female set. 

# Rank (1 = lowest occurance, 4 = highest occurance in that row)
colsToRank = ['DP02_0002E', 'DP02_0004E', 'DP02_0006E', 'DP02_0010E']
newName = ['marriedRank', 'cohabitingRank', 'maleHouseholderRank', 'femaleHouseholderRank']
dp02_table[newName] = dp02_table[colsToRank].rank(axis=1, method='dense')

#Very Low/Low/Medium/High/Very High Bins based on each column
dp02_table['marriedBins'] = pd.qcut(dp02_table['married_P'], q=5, labels = ['Very Low','Low', 'Medium', 'High', 'Very High']) #Based on the distribution within Married set (column)
dp02_table['cohabitingBins'] = pd.qcut(dp02_table['cohabiting_P'], q=5, labels = ['Very Low','Low', 'Medium', 'High', 'Very High']) #Based on the distribution within Cohabiting set and not set values
dp02_table['maleHouseholderBins'] = pd.qcut(dp02_table['maleHouseholder_P'], q=5, labels = ['Very Low','Low', 'Medium', 'High', 'Very High']) #Based on the distribution within Male set and not set values
dp02_table['femaleHouseholderBins'] = pd.qcut(dp02_table['femaleHouseholder_P'], q=5, labels = ['Very Low','Low', 'Medium', 'High', 'Very High']) #Based on the distribution Female set and not set values

# dp02_table.head()

In [15]:
##### Household2
#           Never married   married     separated   widowed     divorced    Total
# Male      DP02_0026E	    DP02_0027E	DP02_0028E	DP02_0029E	DP02_0030E  DP02_0025E
# Female    DP02_0032E	    DP02_0033E	DP02_0034E	DP02_0035E	DP02_0036E  DP02_0031E

# Percents
total = dp02_table2['DP02_0025E'].astype(int) + dp02_table2['DP02_0031E'].astype(int)
dp02_table2['neverMarried_P'] = round(((dp02_table2['DP02_0026E'] + dp02_table2['DP02_0032E']) / total)*100, 2) #Based on distribution within geo_id
dp02_table2['nowMarried_P'] = round(((dp02_table2['DP02_0027E'] + dp02_table2['DP02_0033E']) / total)*100, 2) #Based on distribution within geo_id
dp02_table2['separated_P'] = round(((dp02_table2['DP02_0028E'] + dp02_table2['DP02_0034E']) / total)*100, 2) #Based on distribution within geo_id
dp02_table2['widowed_P'] = round(((dp02_table2['DP02_0029E'] + dp02_table2['DP02_0035E']) / total)*100, 2) #Based on distribution within geo_id
dp02_table2['divorced_P'] = round(((dp02_table2['DP02_0030E'] + dp02_table2['DP02_0036E']) / total)*100, 2) #Based on distribution within geo_id

# Binary
dp02_medianNeverMarried = np.median((dp02_table2['DP02_0026E'] + dp02_table2['DP02_0032E']))
dp02_table2['belowNeverMarriedMedian'] = ((dp02_table2['DP02_0026E'] + dp02_table2['DP02_0032E']) <= dp02_medianNeverMarried).astype(int)
dp02_table2['aboveNeverMarriedMedian'] = ((dp02_table2['DP02_0026E'] + dp02_table2['DP02_0032E']) >= dp02_medianNeverMarried).astype(int)
print(f'Median Never Married: {dp02_medianNeverMarried}')

## Potentially use 5 bins (Very low, low, medium, high, very high) on all?
dp02_table2['neverMarriedBins'] = pd.qcut(dp02_table2['neverMarried_P'], q=5, labels = ['Very Low','Low', 'Medium', 'High', 'Very High']) #Based on the distribution within Never set (column)
dp02_table2['nowMarriedBins'] = pd.qcut(dp02_table2['nowMarried_P'], q=5, labels = ['Very Low','Low', 'Medium', 'High', 'Very High']) #Based on the distribution within Now Married set (column)
dp02_table2['separatedBins'] = pd.qcut(dp02_table2['separated_P'], q=5, labels = ['Very Low','Low', 'Medium', 'High', 'Very High']) #Based on the distribution within Separated set (column)
dp02_table2['widowedBins'] = pd.qcut(dp02_table2['widowed_P'], q=5, labels = ['Very Low','Low', 'Medium', 'High', 'Very High']) #Based on the distribution within Widowed set (column)
dp02_table2['divorcedBins'] = pd.qcut(dp02_table2['divorced_P'], q=5, labels = ['Very Low','Low', 'Medium', 'High', 'Very High']) #Based on the distribution within divorced set (column)

# dp02_table2.head()

Median Never Married: 998.0


In [16]:
##### Household3
# Total population - DP02_0088E
# Native Born - DP02_0089E
# Foreign Born - DP02_0094E

# Percents
dp02_table3['nativeBorn_P'] = round((dp02_table3['DP02_0089E'] / dp02_table3['DP02_0088E'])*100, 2)
dp02_table3['foreignBorn_P'] = round((dp02_table3['DP02_0094E'] / dp02_table3['DP02_0088E'])*100, 2)

# Binary
dp02_nativeBornMedian = np.median(dp02_table3['DP02_0089E'])
dp02_table3['belowNativeBornMedian'] = ((dp02_table3['DP02_0089E']) <= dp02_nativeBornMedian).astype(int)
dp02_table3['aboveNativeBornMedian'] = ((dp02_table3['DP02_0089E']) >= dp02_nativeBornMedian).astype(int)
print(f'Median Native Born: {dp02_nativeBornMedian}')

dp02_foreignBornMedian = np.median(dp02_table3['DP02_0094E'])
dp02_table3['belowForeignBornMedian'] = ((dp02_table3['DP02_0094E']) <= dp02_foreignBornMedian).astype(int)
dp02_table3['aboveForeignBornMedian'] = ((dp02_table3['DP02_0094E']) >= dp02_foreignBornMedian).astype(int)
print(f'Median Foreign Born: {dp02_foreignBornMedian}')

## Potentially use 5 bins (Very low, low, medium, high, very high) on each
dp02_table3['nativeBornBins'] = pd.qcut(dp02_table3['nativeBorn_P'], q=5, labels = ['Very Low','Low', 'Medium', 'High', 'Very High']) #Based on the distribution within native born % set (column)
dp02_table3['foreignBornBins'] = pd.qcut(dp02_table3['foreignBorn_P'], q=5, labels = ['Very Low','Low', 'Medium', 'High', 'Very High']) #Based on the distribution within foreign born % set (column)

# dp02_table3.head()

Median Native Born: 3418.0
Median Foreign Born: 253.0


In [17]:
##### Household4
# Civilian Population 18 and older - DP02_0069E
# Civilan Veterans - DP02_0070E

# Percents
dp02_table4['civilVet_P'] =  round((dp02_table4['DP02_0070E'] / dp02_table4['DP02_0069E'])*100, 2)

# Binary
dp02_vetMedian = np.median(dp02_table4['DP02_0070E'])
dp02_table4['belowCivilVetMedian'] = ((dp02_table4['DP02_0070E']) <= dp02_vetMedian).astype(int)
dp02_table4['aboveCivilVetMedian'] = ((dp02_table4['DP02_0070E']) >= dp02_vetMedian).astype(int)
print(f'Median Civilian Veterans: {dp02_vetMedian}')

## Potentially use 5 bins (Very low, low, medium, high, very high) on vets
dp02_table4['civilVetBins'] =  pd.qcut(dp02_table4['civilVet_P'], q=5, labels = ['Very Low','Low', 'Medium', 'High', 'Very High']) #Based on the distribution within civilVet % set (column)

# dp02_table4.head()

Median Civilian Veterans: 190.0


In [18]:
##### Poverty
# 'B17001_001E' - Total
# 'B17001_002E' - Number of people whose income in past 12 months below poverty level

# Percents
b17001_table['belowPoverty_P'] = round((b17001_table['B17001_002E'] / b17001_table['B17001_001E'])*100, 2)  #Based on distribution within the geo_id (row)
b17001_table['atOrAbovePoverty_P'] = 100 - b17001_table['belowPoverty_P']   #Based on distribution within the geo_id (row)

# Binary
b17001_medianBelowPoverty = np.median(b17001_table['B17001_002E'])
b17001_table['belowPovertyMedian'] = (b17001_table['B17001_002E'] <= b17001_medianBelowPoverty).astype(int)
b17001_table['abovePovertyMedian'] = (b17001_table['B17001_002E'] >= b17001_medianBelowPoverty).astype(int)
print(f"Median Number of People Below Poverty: {b17001_medianBelowPoverty}")

# #Bins
b17001_table['belowPovertyBins'] = pd.qcut(b17001_table['belowPoverty_P'], q=5, labels = ['Very Low', 'Low', 'Medium', 'High', 'Very High']) #Based on the distribution within belowPoverty % set (column)
b17001_table['atOrAbovePovertyBins'] = pd.qcut(b17001_table['atOrAbovePoverty_P'], q=5, labels = ['Very Low', 'Low', 'Medium', 'High', 'Very High']) #Based on the distribution within atOrAbovePoverty % set (column)

b17001_table.head()

Median Number of People Below Poverty: 285.0


,GEO_ID,B17001_001E,B17001_002E,belowPoverty_P,atOrAbovePoverty_P,belowPovertyMedian,abovePovertyMedian,belowPovertyBins,atOrAbovePovertyBins
0,1400000US08001007801,4110,1208,29.39,70.61,0,1,Very High,Very Low
1,1400000US08001007802,4070,1208,29.68,70.32,0,1,Very High,Very Low
2,1400000US08001007900,6173,1112,18.01,81.99,0,1,Very High,Very Low
3,1400000US08001008000,5667,621,10.96,89.04,0,1,High,Low
4,1400000US08001008100,1404,542,38.60,61.40,0,1,Very High,Very Low


In [19]:
##### Age
# S0101_C01_001E - Total population
# S0101_C01_002E - Under 5 years
# S0101_C01_003E - 5 to 9 years
# S0101_C01_004E - 10 to 14 years
# S0101_C01_005E - 15 to 19 years
# S0101_C01_006E - 20 to 24 years
# S0101_C01_007E - 25 to 29 years
# S0101_C01_008E - 30 to 34 years
# S0101_C01_009E - 35 to 39 years
# S0101_C01_010E - 40 to 44 years
# S0101_C01_011E - 45 to 49 years
# S0101_C01_012E - 50 to 54 years
# S0101_C01_013E - 55 to 59 years
# S0101_C01_014E - 60 to 64 years
# S0101_C01_015E - 65 to 69 years
# S0101_C01_016E - 70 to 74 years
# S0101_C01_017E - 75 to 79 years
# S0101_C01_018E - 80 to 84 years
# S0101_C01_019E - 85 years and over

# #Percents
s0101_table['65andOver_P'] = round((( (s0101_table['S0101_C01_015E'] + s0101_table['S0101_C01_016E'] + s0101_table['S0101_C01_017E'] + s0101_table['S0101_C01_018E'] + s0101_table['S0101_C01_019E']) / 
                                  s0101_table['S0101_C01_001E'] ) * 100), 2)
s0101_table['75andOver_P'] = round((( (s0101_table['S0101_C01_017E'] + s0101_table['S0101_C01_018E'] + s0101_table['S0101_C01_019E']) / s0101_table['S0101_C01_001E'] ) * 100), 2) 

# #Bins
s0101_table['65andOverBins'] = pd.qcut(s0101_table['65andOver_P'], q=5, labels = ['Very Low', 'Low', 'Medium', 'High', 'Very High']) #Based on the distribution within 65 and Over Age % set (column)
s0101_table['75andOverBins'] = pd.qcut(s0101_table['75andOver_P'], q=5, labels = ['Very Low', 'Low', 'Medium', 'High', 'Very High']) #Based on the distribution within 75 and Over Age % set (column)

s0101_table.head()

,GEO_ID,S0101_C01_001E,S0101_C01_002E,S0101_C01_003E,S0101_C01_004E,S0101_C01_005E,S0101_C01_006E,S0101_C01_007E,S0101_C01_008E,S0101_C01_009E,...,S0101_C01_014E,S0101_C01_015E,S0101_C01_016E,S0101_C01_017E,S0101_C01_018E,S0101_C01_019E,65andOver_P,75andOver_P,65andOverBins,75andOverBins
0,1400000US08001007801,4145,262,509,300,396,408,320,298,435,...,127,73,51,38,10,10,4.39,1.40,Very Low,Very Low
1,1400000US08001007802,4092,448,328,197,254,386,262,370,385,...,105,70,180,66,10,0,7.97,1.86,Very Low,Very Low
2,1400000US08001007900,6173,560,424,385,657,401,564,575,583,...,248,259,113,21,2,26,6.82,0.79,Very Low,Very Low
3,1400000US08001008000,5701,341,363,303,567,354,416,385,642,...,231,315,155,245,113,11,14.72,6.47,Medium,High
4,1400000US08001008100,1615,16,0,0,22,249,368,317,92,...,86,80,38,53,15,39,13.93,6.63,Medium,High


### Check ACS Tables

In [20]:
## Check Columns
print(f'{s1903_table.columns}\n')
print(f'{s1501_table.columns}\n')
print(f'{dp02_table.columns}\n')
print(f'{dp02_table2.columns}\n')
print(f'{dp02_table3.columns}\n')
print(f'{dp02_table4.columns}\n')
print(f'{b17001_table.columns}\n')
print(f'{s0101_table.columns}\n')

Index(['GEO_ID', 'S1903_C03_001E', 'householdIncomeAboveMedian',
       'householdIncomeBins'],
      dtype='object')

Index(['GEO_ID', '001E_006E', '002E_007E', '008E', '003E_009E',
       '004E_010E_011E', '005E_012E', '013E', 'lessThanHS_P', 'HSNoDiploma_P',
       'HSGrad_P', 'SomeCollegeNoDeg_P', 'BachOrHigher_P', 'GradOrProf_P',
       'noHSDegree_P', 'hasHSDegree_P', 'hsDegreeBins'],
      dtype='object')

Index(['GEO_ID', 'DP02_0001E', 'DP02_0002E', 'DP02_0004E', 'DP02_0006E',
       'DP02_0010E', 'married_P', 'cohabiting_P', 'maleHouseholder_P',
       'femaleHouseholder_P', 'marriedRank', 'cohabitingRank',
       'maleHouseholderRank', 'femaleHouseholderRank', 'marriedBins',
       'cohabitingBins', 'maleHouseholderBins', 'femaleHouseholderBins'],
      dtype='object')

Index(['GEO_ID', 'DP02_0025E', 'DP02_0031E', 'DP02_0026E', 'DP02_0027E',
       'DP02_0028E', 'DP02_0029E', 'DP02_0030E', 'DP02_0032E', 'DP02_0033E',
       'DP02_0034E', 'DP02_0035E', 'DP02_0036E', 'neverMarr

In [21]:
## Check Length
print(f's1903 - {len(s1903_table)}')
print(f's1501 - {len(s1501_table)}')
print(f'dp02 T1 - {len(dp02_table)}')
print(f'dp02 T2 - {len(dp02_table2)}')
print(f'dp02 T3 - {len(dp02_table3)}')
print(f'dp02 T4 - {len(dp02_table4)}')
print(f'b17001 - {len(b17001_table)}')
print(f's0101 - {len(s0101_table)}')

s1903 - 1423
s1501 - 1447
dp02 T1 - 1447
dp02 T2 - 1447
dp02 T3 - 1447
dp02 T4 - 1447
b17001 - 1447
s0101 - 1447


In [22]:
# Table Subsets - Bins
s1903_subset = s1903_table[['GEO_ID', 'householdIncomeBins']]
s1501_subset = s1501_table[['GEO_ID', 'hsDegreeBins']]
dp02_subset = dp02_table[['GEO_ID', 'marriedBins', 'cohabitingBins', 'maleHouseholderBins', 'femaleHouseholderBins']]
dp02_subset2 = dp02_table2[['GEO_ID', 'neverMarriedBins', 'nowMarriedBins', 'separatedBins', 'widowedBins', 'divorcedBins']]
dp02_subset3 = dp02_table3[['GEO_ID', 'nativeBornBins', 'foreignBornBins']]
dp02_subset4 = dp02_table4[['GEO_ID', 'civilVetBins']]
b17001_subset = b17001_table[['GEO_ID', 'belowPovertyBins', 'atOrAbovePovertyBins']]
s0101_subset = s0101_table[['GEO_ID', '65andOverBins', '75andOverBins']]

### Merge ACS Tables

In [23]:
# MERGE ACS TABLES
from functools import reduce

tables = [s1903_subset, s1501_subset, dp02_subset, dp02_subset2, dp02_subset3, dp02_subset4, b17001_subset, s0101_subset]
acsTable = reduce(lambda left, right: pd.merge(left, right, on="GEO_ID", how='inner'), tables)

acsTable

,GEO_ID,householdIncomeBins,hsDegreeBins,marriedBins,cohabitingBins,maleHouseholderBins,femaleHouseholderBins,neverMarriedBins,nowMarriedBins,separatedBins,widowedBins,divorcedBins,nativeBornBins,foreignBornBins,civilVetBins,belowPovertyBins,atOrAbovePovertyBins,65andOverBins,75andOverBins
0,1400000US08001007801,Very Low,Very Low,Very Low,Very High,Very High,Very High,Very High,Very Low,Very High,High,High,Very Low,Very High,Very Low,Very High,Very Low,Very Low,Very Low
1,1400000US08001007802,Very Low,Very Low,Very Low,Very High,Very High,Medium,Very High,Very Low,Very Low,Low,High,Very Low,Very High,Medium,Very High,Very Low,Very Low,Very Low
2,1400000US08001007900,Low,Very Low,Very Low,Very High,High,Medium,Very High,Very Low,Very High,Medium,High,Very Low,Very High,Very Low,Very High,Very Low,Very Low,Very Low
3,1400000US08001008000,Low,Very Low,Low,Medium,High,Very High,High,Low,Very High,Medium,Medium,Very Low,Very High,High,High,Low,Medium,High
4,1400000US08001008100,Very Low,Very High,Very Low,Very High,Very High,Very High,Very High,Very Low,Low,High,High,Very Low,Very High,High,Very High,Very Low,Medium,High
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1418,1400000US08123002300,Medium,Very Low,High,Very Low,Medium,Low,Very Low,High,Medium,High,Medium,Medium,Medium,Medium,Medium,Medium,Medium,High
1419,1400000US08123002501,Low,Medium,High,Low,Very Low,Medium,Low,High,Medium,Medium,Low,Medium,Medium,High,Very High,Very Low,Medium,Medium
1420,1400000US08123002502,Medium,Very Low,High,Low,Low,Low,Low,High,High,High,Low,Low,High,High,High,Low,Low,Low
1421,1400000US08125963100,Low,Very Low,High,Low,Low,High,Low,High,High,High,Low,Low,High,Medium,High,Low,High,High


### ACS Table Columns

| Output Column Header   | ACS Columns Utilized |
| -------- | ------- |
| GEO_ID   | GEO_ID |
| householdIncomeBins   | S1903_C03_001E |
| hsDegreeBins   | S1501_C01_001E (total 18-24) <br> S1501_C01_003E (High school graduate (18-24)) <br> S1501_C01_004E (Some college or associates degree (18-24)) <br> S1501_C01_005E (Bachelor's degree or higher (18-24)) <br> S1501_C01_006E (total 25+) <br> S1501_C01_008E (9th to 12th, no diploma (25+)) <br> S1501_C01_009E (High school graduate (25+)) <br> S1501_C01_010E (Some college, no degree (25+)) <br> S1501_C01_011E (Associates Degree (25+)) <br> S1501_C01_012E (Bachelor's degree (25+)) <br> S1501_C01_013E (graduate or professional degree (25+)) |
| marriedBins  | DP02_0001 (total) <br> DP02_0002E (married couple) |
| cohabitingBins   | DP02_0001 (total) <br> DP02_0004E (cohabiting couple)  |
| maleHouseholderBins   | DP02_0001 (total) <br> DP02_0006E (male householder no spouse)  |
| femaleHouseholderBins   | DP02_0001 (total) <br> DP02_0010E (female householder no spouse)  |
| neverMarriedBins | DP02_0025E (Male Total) <br> DP02_0031E (Female Total) <br> DP02_0026E (Male Never Married) <br> DP02_0032E (Female Never Married) |
| nowMarriedBins   | DP02_0025E (Male Total) <br> DP02_0031E (Female Total) <br> DP02_0027E (Male Now Married) <br> DP02_0033E (Female Now Married) |
| separatedBins   | DP02_0025E (Male Total) <br> DP02_0031E (Female Total) <br> DP02_0028E (Male Separated) <br> DP02_0034E (Female Separated) |
| widowedBins   | DP02_0025E (Male Total) <br> DP02_0031E (Female Total) <br> DP02_0029E (Male Widowed) <br> DP02_0035E (Female Widowed) |
| divorcedBins  | DP02_0025E (Male Total) <br> DP02_0031E (Female Total) <br> DP02_0030E (Male Divorced) <br> DP02_0036E (Female Divorced) |
| nativeBornBins   | DP02_0088E (total) <br> DP02_0089E (Native Born)|
| foreignBornBins   | DP02_0088E (total) <br> DP02_0094E (Foreign Born) |
| civilVetBins   | DP02_0069E (Civilian Population 18+) <br> DP02_0070E (Civilan Veterans) |
| belowPovertyBins  | B17001_001E (total) <br> B17001_002E (Number of people whose income below poverty level in last 12months) |
| atOrAbovePovertyBins  | B17001_001E (total) <br> B17001_002E (Number of people whose income below poverty level in last 12months) |
| 65andOverBins  | S0101_C01_001E (total) <br> S0101_C01_015E (age 65-69) <br> S0101_C01_016E (age 70-) <br> S0101_C01_017E (age 75-79) <br> S0101_C01_018E (age 80-84) <br> S0101_C01_019E (age 85 and over) |
| 75andOverBins  | S0101_C01_001E (total) <br> S0101_C01_017E (age 75-79) <br> S0101_C01_018E (age 80-84) <br> S0101_C01_019E (age 85 and over) |


### Pull In and Add NPI Hospice Provider Information

In [24]:
####################################################################
## TEST DATA
# testData = {'NPI': ['1760093470', '1861097982', '1447936901', '1043826795'], 
#             'zipPlusFour': ['80237-2857', '81008-2130', '80111-2213', '80111-7957'],    ##Last zipcode is not accurate - just used for testing
#             'taxType': ['NPI-2 Organization', 'NPI-2 Organization', 'NPI-2 Organization', 'NPI-2 Organization'], 
#             'primaryTax': ['Yes', 'Yes', 'Yes', 'No'], 
#             'Issuer': ['MEDICAID', '', '', ''], 
#             'status': ['Active', 'Active', 'Active', 'Active']}

# ## CREATE TEST DF
# df = pd.DataFrame(testData)

####################################################################

In [25]:
### Import NPI Provider Information
df = pd.read_pickle('df.pkl') 

In [26]:
from zipcodeToCensusTract import convertPostalToLatLong, convertLatLongToCensusTract

In [28]:
df

,NPI,Name,Status,Location State,Primary Practice Address,Taxonomy Entries,Issuers,ZIP9,ZIP5
0,1760093470,247 HOME HEALTH CARE LTD,A,CO,"8055 E TUFTS AVE STE 250 , DENVER, CO 802372857","[(Hospice Care, Community Based, True)]",[],802372857,80237
1,1740072065,A BETTER COLORADO HOSPICE LLC,A,CO,"126 W D ST STE 200 , PUEBLO, CO 810034430","[(Hospice Care, Community Based, True)]",[],810034430,81003
2,1003483330,"A PEACEFUL JOURNEY HOSPICE, LLC",A,CO,"2851 S PARKER RD STE 1130 , AURORA, CO 800142732","[(Hospice Care, Community Based, True)]",[],800142732,80014
3,1861097982,"ABODE HEALTHCARE COLORADO, INC",A,CO,"1050 EAGLERIDGE BLVD , PUEBLO, CO 810082130","[(Hospice Care, Community Based, True)]",[],810082130,81008
4,1326459025,"ABODE HEALTHCARE COLORADO, INC",A,CO,"5465 MARK DABLING BLVD , COLORADO SPRINGS, CO ...","[(Hospice Care, Community Based, True)]",[],809183842,80918
...,...,...,...,...,...,...,...,...,...
294,1518171834,WOMENS HEALTH CENTER INC,A,CO,"1600 N GRAND AVE STE 400 , PUEBLO, CO 810032760","[(Obstetrics & Gynecology, Hospice and Palliat...",[],810032760,81003
298,1013648583,YNA HOSPICE INC,A,CO,"3190 S VAUGHN WAY STE 550 OFF 520 , AURORA, CO...","[(Hospice Care, Community Based, True)]",[],80014,80014
299,1285645382,YULIYA GOSTISHCHEVA,A,CO,"1240 S PARKER RD #106, DENVER, CO 802317558","[(Hospice Care, Community Based, True)]",[],802317558,80231
300,1942931415,ZA HOSPICE INC,A,CO,"102 S TEJON ST STE 1100 OFF 1111 , COLORADO SP...","[(Hospice Care, Community Based, True)]",[],80903,80903


In [29]:
# ## Create zipcode column
# df['zip'] = df['zipPlusFour'].str[:5]

## Call function and add lat/long to df
df[['lat', 'long']] = df['ZIP5'].apply(lambda zip: pd.Series(convertPostalToLatLong(zip)))

In [30]:
## Call function and add censusTract to df
df['censusTract'] = df.apply(lambda row: convertLatLongToCensusTract(row['lat'], row['long']), axis = 1)
df

,NPI,Name,Status,Location State,Primary Practice Address,Taxonomy Entries,Issuers,ZIP9,ZIP5,lat,long,censusTract
0,1760093470,247 HOME HEALTH CARE LTD,A,CO,"8055 E TUFTS AVE STE 250 , DENVER, CO 802372857","[(Hospice Care, Community Based, True)]",[],802372857,80237,39.6431,-104.8987,1400000US08031006816
1,1740072065,A BETTER COLORADO HOSPICE LLC,A,CO,"126 W D ST STE 200 , PUEBLO, CO 810034430","[(Hospice Care, Community Based, True)]",[],810034430,81003,38.2843,-104.6234,1400000US08101000400
2,1003483330,"A PEACEFUL JOURNEY HOSPICE, LLC",A,CO,"2851 S PARKER RD STE 1130 , AURORA, CO 800142732","[(Hospice Care, Community Based, True)]",[],800142732,80014,39.6662,-104.8350,1400000US08005080400
3,1861097982,"ABODE HEALTHCARE COLORADO, INC",A,CO,"1050 EAGLERIDGE BLVD , PUEBLO, CO 810082130","[(Hospice Care, Community Based, True)]",[],810082130,81008,38.3133,-104.6284,1400000US08101002919
4,1326459025,"ABODE HEALTHCARE COLORADO, INC",A,CO,"5465 MARK DABLING BLVD , COLORADO SPRINGS, CO ...","[(Hospice Care, Community Based, True)]",[],809183842,80918,38.9129,-104.7734,1400000US08041004800
...,...,...,...,...,...,...,...,...,...,...,...,...
294,1518171834,WOMENS HEALTH CENTER INC,A,CO,"1600 N GRAND AVE STE 400 , PUEBLO, CO 810032760","[(Obstetrics & Gynecology, Hospice and Palliat...",[],810032760,81003,38.2843,-104.6234,1400000US08101000400
298,1013648583,YNA HOSPICE INC,A,CO,"3190 S VAUGHN WAY STE 550 OFF 520 , AURORA, CO...","[(Hospice Care, Community Based, True)]",[],80014,80014,39.6662,-104.8350,1400000US08005080400
299,1285645382,YULIYA GOSTISHCHEVA,A,CO,"1240 S PARKER RD #106, DENVER, CO 802317558","[(Hospice Care, Community Based, True)]",[],802317558,80231,39.6793,-104.8843,1400000US08005087200
300,1942931415,ZA HOSPICE INC,A,CO,"102 S TEJON ST STE 1100 OFF 1111 , COLORADO SP...","[(Hospice Care, Community Based, True)]",[],80903,80903,38.8388,-104.8145,1400000US08041002200


In [31]:
## Get count of NPI Providers per CensusTract
providerCounts = df['censusTract'].value_counts().reset_index()
providerCounts.columns = ['GEO_ID', 'Provider Count']
providerCounts

,GEO_ID,Provider Count
0,1400000US08005006857,23
1,1400000US08005080400,15
2,1400000US08101000400,8
3,1400000US08031006816,7
4,1400000US08005006709,7
...,...,...
93,1400000US08031003205,1
94,1400000US08013013601,1
95,1400000US08063962100,1
96,1400000US08099000300,1


In [32]:
## Merge Count of Providers per CensusTract with ACS Data information
mergedTable = pd.merge(providerCounts, acsTable, on = 'GEO_ID', how = 'left')
mergedTable

,GEO_ID,Provider Count,householdIncomeBins,hsDegreeBins,marriedBins,cohabitingBins,maleHouseholderBins,femaleHouseholderBins,neverMarriedBins,nowMarriedBins,separatedBins,widowedBins,divorcedBins,nativeBornBins,foreignBornBins,civilVetBins,belowPovertyBins,atOrAbovePovertyBins,65andOverBins,75andOverBins
0,1400000US08005006857,23,Very High,Very High,Very High,Very Low,Medium,Very Low,Very Low,Very High,Medium,Very Low,Very Low,Very Low,Very High,Medium,Very Low,Very High,Very High,Very High
1,1400000US08005080400,15,Medium,High,Low,High,High,High,Medium,Low,Very High,High,Very High,Low,High,Medium,Medium,Medium,High,High
2,1400000US08101000400,8,Very Low,Very Low,Very Low,Low,Very High,Very High,Medium,Very Low,Very High,High,Very High,High,Low,Low,Very High,Very Low,Medium,High
3,1400000US08031006816,7,Very Low,Medium,Low,High,Very High,Medium,High,Low,Low,High,Very High,Low,High,Very High,High,Low,Very High,Very High
4,1400000US08005006709,7,Very High,High,High,Low,Medium,Low,Low,High,Medium,Low,High,Low,High,Medium,Very Low,Very High,Medium,High
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,1400000US08031003205,1,Low,Very High,Very Low,High,Very High,High,Very High,Low,Very Low,Low,Medium,Medium,Medium,Medium,High,Low,High,Very High
94,1400000US08013013601,1,High,High,High,High,Low,Low,Very Low,High,High,High,Medium,High,Low,Medium,Medium,Medium,High,Medium
95,1400000US08063962100,1,Very Low,Low,Medium,Low,Medium,High,Low,Medium,Very High,Very High,Medium,Medium,Medium,Medium,Medium,Medium,High,High
96,1400000US08099000300,1,Very Low,Very Low,Medium,Low,Medium,Medium,Medium,Medium,Medium,Very High,Low,Low,High,High,Very High,Very Low,Medium,High


In [33]:
## Check length of mergedTable
len(mergedTable)

98

In [34]:
## Save DF with to_pickle so it can be used in another file
mergedTable.to_pickle('mergedTable.pkl')